Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All) to avoid typical problems with Jupyter notebooks. **Unfortunately, this does not work with Chrome right now, you will also need to reload the tab in Chrome afterwards**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Please put your name here:

In [5]:
NAME = "Aymane Hachcham"

---

# Latent Dirichlet Allocation

Now we will use latent dirichlet allocation

**Important notice:** the "Validate" function might timeout at 30 or 60 seconds.
We intend to have the actual autograding later run with higher tolerance.

In [6]:
### Load the input data - do not modify
import json, gzip, numpy as np
raw = json.load(gzip.open("/data/simpsonswiki.json.gz", "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["c"] for x in raw]

In [7]:
### This cell reduces the data set size for the autograder tests - do not modify

In [8]:
### Vectorize the text - do not modify
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words="english", min_df=5)
counts = cvect.fit_transform(texts)
vocabulary = cvect.get_feature_names_out()

## Explore your result

Explore the result: write a function to determine the most important words for each factor, and the most relevant documents.

**COPY your code from the first file here** (one of the rare cases, where copying is okay)

In [9]:
def most_important(vocabulary, factor, k=10):
    """Most important words for each factor"""
    indices_max_values = np.argpartition(factor, -k)[-k:]
    list_vocabs = [vocabulary[i] for i in indices_max_values]
    return list_vocabs

def most_relevant(assignment, k=5):
    """Most relevant documents for each factor (return document indexes)"""
    indices_max_values = np.argpartition(assignment, -k)[-k:]
    return indices_max_values

def explain(vocabulary, titles=None, classes=None, factors=None, assignment=None, weights=None):
    """Print an explanation for each factor.
       If weights is None, use the relative share of the assignment weights.
       Print the ARI when assigning each document to its maximum only."""
    from sklearn.metrics import adjusted_rand_score
    for i, f in enumerate(factors):
        print('For the Factor: {}, these are the following results'.format(i))
        important_vocabs = most_important(vocabulary, f)
        print('The most relevant words in this topic are: ')
        print('-------------------------------------------------------')
        print('\n')
        print(important_vocabs)
        important_docs = most_relevant(assignment)
        print('-------------------------------------------------------')
        print('\n')
        print('The most relevant documents belonging to this topic are: ')
        print([titles[i] for fact in important_docs for i in fact])
        print('\n')
        print('Their respective classes are ')
        print([classes[i] for fact in important_docs for i in fact])
        if weights is not None:
            factor_weight = weights[i]
            print('-------------------------------------------------------')
            print('\n')
            print('The Weight factor for this topic is {}'.format(factor_weight))
        print('#################################################################')

## Latent Dirichlet Allocation

Apply LDA from the `sklearn` package onto the data. Return the topic word distributions as well as the document topic distributions.

In [10]:
#%%
### Use sklearn LDA here:
def lda(counts, k):
    """Latent Dirichlet Allocation. Return the factors and document assignment"""
    # YOUR CODE HERE
    from sklearn.decomposition import LatentDirichletAllocation
    model  = LatentDirichletAllocation(n_components=k)

    assignment = model.fit_transform(counts)
    topic_term_matrix = model.components_
    factors = np.apply_along_axis(lambda x: x/x.sum(), 1, topic_term_matrix)

    return factors, assignment

In [11]:
### Automatic tests. You do not need to understand or modify this code.
_tmp = lda(counts[:100], 2)
assert len(_tmp) == 2, "Incomplete result"
assert _tmp[0].shape == (2, counts.shape[1]), "Factor shape is not correct."
assert _tmp[1].shape == (100, 2), "Assignment shape is not correct."
assert abs(_tmp[0].sum()-2)<1e-7, "Topic word matrix are not probabilities."
assert abs(_tmp[1].sum()-100)<1e-7, "Document topic matrix are not probabilities."
del _tmp

In [12]:
# Explore your result. These must be meaningful topics!
# but the most relevant documents found here are not as good
lda_factors, lda_assignment = lda(counts, 6)
explain(vocabulary, titles, classes, lda_factors, lda_assignment)

KeyboardInterrupt: 

In [ ]:
### This cell contains additional tests. You do not need to modify this cell.